# Linear Discriminant Analysis, Quadratic Discriminant Analysis, KNN and Logistics

## Problem Statement

#### Objective :


### Dataseet Information 

This datset is taken from An introduction to statistical learning.  
Year : The year that the observation was recorded.  
Lag1 : Percentage return for previous week.  
Lag2 : Percentage return for 2 weeks previous.   
Lag3 : Percentage return for 3 weeks previous.  
Lag4 : Percentage return for 4 weeks previous.  
Lag5 : Percentage return for 5 weeks previous.  
Volume : Volume of Shares traded(average number of daily shares traded in billions).  
Today :A factor with levels Down and Up indicating weather the market had positive or negative return on a given week. 
In our dataset 1089 rows and 9 columns.

### Importing the required libraries

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from warnings import filterwarnings
filterwarnings("ignore")
import sympy as sym    
import scipy.stats as ss

### Importing the Dataset

In [2]:

weekly=pd.read_csv("weekly_data.csv")
weekly.head()

,Year,Lag1,Lag2,Lag3,Lag4,Lag5,Volume,Today,Direction
0,1990,0.816,1.572,-3.936,-0.229,-3.484,0.154976,-0.270,Down
1,1990,-0.270,0.816,1.572,-3.936,-0.229,0.148574,-2.576,Down
2,1990,-2.576,-0.270,0.816,1.572,-3.936,0.159837,3.514,Up
3,1990,3.514,-2.576,-0.270,0.816,1.572,0.161630,0.712,Up
4,1990,0.712,3.514,-2.576,-0.270,0.816,0.153728,1.178,Up


In [3]:
weekly.corr(method='pearson')

,Year,Lag1,Lag2,Lag3,Lag4,Lag5,Volume,Today
Year,1.000000,-0.032289,-0.033390,-0.030006,-0.031128,-0.030519,0.841942,-0.032460
Lag1,-0.032289,1.000000,-0.074853,0.058636,-0.071274,-0.008183,-0.064951,-0.075032
Lag2,-0.033390,-0.074853,1.000000,-0.075721,0.058382,-0.072499,-0.085513,0.059167
Lag3,-0.030006,0.058636,-0.075721,1.000000,-0.075396,0.060657,-0.069288,-0.071244
Lag4,-0.031128,-0.071274,0.058382,-0.075396,1.000000,-0.075675,-0.061075,-0.007826
Lag5,-0.030519,-0.008183,-0.072499,0.060657,-0.075675,1.000000,-0.058517,0.011013
Volume,0.841942,-0.064951,-0.085513,-0.069288,-0.061075,-0.058517,1.000000,-0.033078
Today,-0.032460,-0.075032,0.059167,-0.071244,-0.007826,0.011013,-0.033078,1.000000


The correlation between the lag variable and today's return are close to zero.  
and also the correlation between the lag variable and volume is close to zero.
The only substantial correlation between year and volume. By plotting the data we see that volume is increasing over time.
The average number of shares traded daily increased from 1990 to 2010.

In [4]:
x_1=weekly['Lag1']
plt.scatter(x_1,y, marker='+',color='red')
plt.title('plot of Lag1')
plt.show()


NameError: name 'y' is not defined

from the above graph, the different values of Lag1 corresponding Direction values are either up and down.

In [ ]:
def categorizing(dat):
    cat=dat.astype('category').cat.codes
    return cat
weekly['Direction']=categorizing(weekly['Direction'])    # We converted Down in 0 and Up in 1

In [ ]:
x=weekly.drop(['Year','Direction','Today'],axis=1).values
y=weekly['Direction']
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.25,random_state=1)
from sklearn.linear_model import LogisticRegression
logmodel=LogisticRegression()
logmodel.fit(x_train,y_train)
y_pred=logmodel.predict(x_test)

In [ ]:
from sklearn.metrics import accuracy_score
Accuracy_score=accuracy_score(y_test,y_pred)
print("Accuracy_score: "+"{:.2f}".format(Accuracy_score));

In [ ]:
import statsmodels.api as sm
x_train_logit=sm.add_constant(x_train)
log_reg=sm.Logit(y_train,x_train_logit).fit()
print(log_reg.summary())

Here we plot summary of Logistic regression.  
Here is All p-value is greater than 0.05.There is no variables contributes significantly to the model.  
The estimated logit is  
$\log\left(\frac{\pi(x)}{1-\pi(x)}\right)$=0.2224-0.0341*Lag1+0.0333*Lag2-0.0187*Lag3-0.0186*Lag4-0.0282*Lag5+0.0067*Volume.

Log-Likelihood - The natural logarithm of the MLE. MLE is the optimization process of finding the set of parameters.  
LL-Null - The value of log-likelihood of the model when no independent variable is included.  
Pseudo R-sq- It is the ratio of 1- log likelihood of the full model to that of the null model. Here is Pseudo R-sq is 0.003993
is less then the model is not good.

**Que-Do any of the predictors appear to be statistically significant?**  
**Ans-There is none of the Predictor which is statistically significant.**  



In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix
c_m=confusion_matrix(y_test,y_pred)
c_m
plot_confusion_matrix(logmodel,x_test,y_test,
                     
                     display_labels=["logistic(Down)","logistic(Up)"])

The diagonal element of the confusion matrix indicate correct predictions, while the off-diagonal reprsent incorrect predictor 
Hence our model is correctly predicted that market would go up on 148 days and that it go down on 11 days ,
for total 11+148=159,correct predictions and 114 are incorrect prediction.

**Odds Ratio**  
from the above summary we got the coeffiecients but using this it is difficult to interpret the odd ratio, 
log odds is not intuitive in its interepretation.Lets convert this odds ratio and interepret the model again.

In [ ]:
model_odds=pd.DataFrame(np.exp(log_reg.params),columns=['Odds Ratio'])
model_odds['z-values']=log_reg.pvalues
model_odds

If the Lag1 is increases by one unit then Direction of market decreases by 4$\%$.  
If the Lag3 is increases by one unit then Direction of market decreases by 2$\%$.  
If the Lag4 is increases by one unit then the Direction of market is decreases by 2$\%$.  
If the Lag 5 is increases by one unit then the Direction of market is decreases by 3$\%$.  
There is no effect of increases or decreases in  Lag2 and Volume on Direction of the market. 

In [ ]:
def categorizing(dat):
    cat=dat.astype('category').cat.codes
    return cat
weekly['Direction']=categorizing(weekly['Direction'])    # We converted Down in 0 and Up in 1

In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
x_1 = weekly[['Lag1','Lag2']]
y_1 = weekly[['Direction']]
lda = LinearDiscriminantAnalysis()
lda.fit(x_1, y_1)

In [ ]:
x_1_train=weekly[['Lag1','Lag2']]
y_1_train=weekly[['Direction']]
x_1_test=weekly[['Lag1','Lag2']]
y_1_test=weekly[['Direction']]
LDA_model=lda.fit(x_1_train,y_1_train)
y_1_pred=lda.predict(x_1_test)
print(LDA_model.priors_)      #Prior Probabilities of group.

LDA output indiacte that $\pi_1$=0.444444 and $\pi_2$=0.555555. In other words probability that 44.44 of the traning observation corrsponding 
to direction during which market went down. And probability 55.55 $\%$ of the traning observation corrsponding to direction during 
which market went up.

In [ ]:
print(LDA_model.means_)   #Group means

The above are the group means,these are the average of each predictor within each class,and are used by LDA as estimate of $\mu_k$ 
.These suggest that there is tendency for the previous 2 days return to be positive when the market increases

In [ ]:
print(LDA_model.coef_)   #coefficient of Discriminant function.

The coeffiecient of linear discriminants output provide the linear combination of Lag1 and Lag2 that are used to form the Decision rule.

If -0.03849002 Lag1+0.05955142 Lag2 will large. The LDA classifier will predict a market increses and if it is small ,then LDA classifier will predict a market down.

In [9]:

from sklearn.metrics import confusion_matrix
matrix=confusion_matrix(y_1_test,y_1_pred)
print(matrix)
plot_confusion_matrix(lda,x_1_test,y_1_test,
                     
                     display_labels=["LDA(Down)","LDA(Up)"])


NameError: name 'y_1_test' is not defined

The diagonal elemets are positively identified by the Direction. The market assigned 37 days to the "Down" class. and 568 days to the "Up" class.

In [ ]:
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
qda = QuadraticDiscriminantAnalysis()
qda.fit(x_1, y_1)
y2_pred=qda.predict(x_1_test)
model2=qda.fit(x_1_train,y_1_train)

In [ ]:
print(model2.priors_)
print(model2.means_)

The output contains the group means. But it does not contain te coefficients of the linear discriminants, because the QDA classifier involves a quadratic, rather than a linear, function of the predictors.

In [ ]:
from sklearn.metrics import confusion_matrix
matrix=confusion_matrix(y_1_test,y2_pred)
print(matrix)
plot_confusion_matrix(qda,x_1_test,y_1_test,
                     
                     display_labels=["QDA(Down)","QDA(Up)"])

In [ ]:
from sklearn.metrics import accuracy_score
Accuracy_score=accuracy_score(y_1_test,y2_pred)
print("Accuracy_score "+"{:.2f}".format(Accuracy_score))     #Mean Accuracy

The QDA predictions are accurate almost 55.37$\%$, LDA prediction are accurate almost 55.55$\%$ and logistic regression prediction score is 58.24$\%$. Then we say that the logistic regression gives more accuracy than LDA and QDA, the level of accuracy is quite impressive for weekly dataset.

In [ ]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
x_1_train_1=sc.fit_transform(x_1_train)
x_1_test=sc.transform(x_1_test)
from sklearn.neighbors import KNeighborsClassifier
knn=KNeighborsClassifier(n_neighbors=1)
knn.fit(x_1_train,y_1_train)

In [ ]:
y_pred_knn=knn.predict(x_1_test)
print(y_pred_knn)

In [ ]:
from sklearn.metrics import confusion_matrix
matrix=confusion_matrix(y_1_test,y_pred_knn)
print(matrix)

In [ ]:
plot_confusion_matrix(knn,x_1_test,y_1_test,
                     
                     display_labels=["KNN(Down)","KNN(Up)"])

The result using k=1 are not very good.The diagonal elements are positively identified by class and tells us that the market goes on down by 219 days and the market increses by 343 days.

In [ ]:
from sklearn.metrics import accuracy_score
Accuracy_score=accuracy_score(y_1_test,y_pred_knn)
print("Accuracy of the model is",Accuracy_score)
print("Error rate of model is",(1-Accuracy_score))

We got Accuracy of the model is 51$\%$.As we have increased the number of neighbors in the model.the accuracy got decreases but this is not necessray for each case.

In [ ]:
# for k =2
from sklearn.neighbors import KNeighborsClassifier
knn1=KNeighborsClassifier(n_neighbors=2)
knn1.fit(x_1_train,y_1_train)
y_pred_knn1=knn1.predict(x_1_test)
from sklearn.metrics import accuracy_score
Accuracy_score=accuracy_score(y_1_test,y_pred_knn1)
print(Accuracy_score)